In [1]:
import qualreas as qr

In [2]:
alg_path = "/Users/alfredreich/Documents/Github/repo/qualreas/Algebras/"
json_file = "/Users/alfredreich/Documents/Github/repo/qualreas/Misc/rcc8_example.json"

In [3]:
net = qr.load_network(alg_path, json_file)
net.summary()


RCC8 Example 1: 5 nodes, 25 edges
  Algebra: RCC8Algebra
  House1:
    => House1: EQ
    => House2: DC
    => Property1: NTPP|TPP
    => Property2: DC|EC
    => Road: EC
  House2:
    => House2: EQ
    => House1: DC
    => Property1: DC|EC
    => Property2: NTPP
    => Road: EC
  Property1:
    => Property1: EQ
    => House1: NTPPI|TPPI
    => House2: DC|EC
    => Property2: DC|EC
    => Road: DC|EC|EQ|NTPP|NTPPI|PO|TPP|TPPI
  Property2:
    => Property2: EQ
    => House1: DC|EC
    => House2: NTPPI
    => Property1: DC|EC
    => Road: DC|EC|EQ|NTPP|NTPPI|PO|TPP|TPPI
  Road:
    => Road: EQ
    => House1: EC
    => House2: EC
    => Property1: DC|EC|EQ|NTPP|NTPPI|PO|TPP|TPPI
    => Property2: DC|EC|EQ|NTPP|NTPPI|PO|TPP|TPPI


In [4]:
net.propagate()
net.summary()


RCC8 Example 1: 5 nodes, 25 edges
  Algebra: RCC8Algebra
  House1:
    => House1: EQ
    => House2: DC
    => Property1: NTPP|TPP
    => Property2: DC|EC
    => Road: EC
  House2:
    => House2: EQ
    => House1: DC
    => Property1: DC
    => Property2: NTPP
    => Road: EC
  Property1:
    => Property1: EQ
    => House1: NTPPI|TPPI
    => House2: DC
    => Property2: DC|EC
    => Road: EC|PO
  Property2:
    => Property2: EQ
    => House1: DC|EC
    => House2: NTPPI
    => Property1: DC|EC
    => Road: PO|TPPI
  Road:
    => Road: EQ
    => House1: EC
    => House2: EC
    => Property1: EC|PO
    => Property2: PO|TPP
